In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential

In [2]:
# layer = layers.Conv2D(4, )

# 更底层的函数
# out = tf.nn.conv2d()

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(2345)


In [4]:
conv_layers = [
    layers.Conv2D(64, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    layers.Conv2D(128, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    layers.Conv2D(256, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),
]

In [9]:
conv_net = Sequential(conv_layers)
conv_net.build(input_shape=[None, 32,32,3])
x = tf.random.normal([4,32,32,3])
out = conv_net(x)
print(out.shape)
conv_net.summary()

(4, 1, 1, 512)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                   

In [10]:
fc_net = Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(100)
])
fc_net.build(input_shape=[None, 512])

def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

In [11]:
from tensorflow.keras import datasets
(x,y), (x_test,y_test)  = datasets.cifar100.load_data()
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)
print(x.shape, y.shape, x_test.shape, y_test.shape)


169017344/169001437 [==============================] - 74s 0us/step
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


In [12]:
train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.map(preprocess).shuffle(10000).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.map(preprocess).batch(64)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [13]:
sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape,
      tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

sample: (128, 32, 32, 3) (128,) tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [15]:
optimizer = tf.optimizers.Adam(1e-4)
for epoch in range(3):
    for step, (x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out = conv_net(x)
            out = tf.reshape(out, [-1, 512])
            logits = fc_net(out)
            y_onehot = tf.one_hot(y, depth=100)
            loss = tf.losses.categorical_crossentropy(y_true=y_onehot, y_pred=logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, conv_net.trainable_variables+fc_net.trainable_variables)
        optimizer.apply_gradients(zip(grads, conv_net.trainable_variables+fc_net.trainable_variables))
        if step %100 == 0:
                print(epoch, step, 'loss:', float(loss))

    total_correct = 0
    total_num = 0
    for x,y in test_db:
        out = conv_net(x)
        out = tf.reshape(out, [-1, 512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits=logits, axis=1)
        pred = tf.argmax(prob)
        pred = tf.cast(pred, dtype=tf.int32)
        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)
        total_num += x.shape[0]
        total_correct += int(correct)
    acc = total_correct/ total_num
    print(epoch, 'acc:', acc)




0 0 loss: 4.605621337890625


KeyboardInterrupt: 

In [16]:
# batchnorm

In [17]:
net = layers.BatchNormalization()
x = tf.random.normal([2,3])
out = net(x)
net.trainable_variables

[<tf.Variable 'batch_normalization/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [18]:
net.variables

[<tf.Variable 'batch_normalization/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_variance:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>]